### Contents
1. React Router Basics
2. Advanced Route Definitions
3. Handling Navigation
4. Routers In Depth


# 1. Router Basics
- Setup your router
  - import a Router and wrap the entire application
- Define your routes
  - Wrap a `Routes` component around your `<Route path="" element={component} />` components 
- Handle navigation
  - Use the `<Link to="" />` component 

### a. Configuring the Router
All you need to do is import the specific router you need (`BrowserRouter` for the web and `NativeRouter` for mobile) and wrap your entire application in that router.

```js
  <BrowserRouter>
    <App />
  </BrowserRouter>
```

Generally you will import your router in the `index.js` page of your application and it will wrap your `App` component. The router works just like a context in React and provides all the necessary information to your application so you can do routing and use all the custom hooks from React Router.

### b. Defining Routes
The next step is to define your routes. This is generally done at the top level of your application, such as in the `App` component, but can be done anywhere you want. Defining routes is as simple as defining a single `Route` component for each route in your application and then putting all those `Route` components in a single `Routes` component

```js
  <Routes>
    <Route path="/" element={<Home />} />
    <Route path="/books" element={<BookList />} />
  </Routes>
```

Whenever your URL changes React Router will look at the routes defined in your `Routes` component and it will render the content in the `element` prop of the `Route` that has a path that matches the URL. In the above example if our URL was `/books` then the `BookList` component would be rendered.

The nice thing about React Router is that when you navigate between pages it will only refresh the content inside your `Routes` component. All the rest of the content on your page will stay the same which helps with performance and user experience.

### c. Handling Navigation

The final step to React Router is handling navigation. Normally in an application you would navigate with anchor tags, but React Router uses its own custom `Link` component to handle navigation. This `Link` component is just a wrapper around an anchor tag that helps ensure all the routing and conditional re-rendering is handled properly so you can use it just like your would a normal anchor tag.

```js
  <Link to="/">Home</Link>
  <Link to="/books">Books</Link>
```

You will also notice that we used the `to` prop to set the URL instead of the `href` prop you are used to using with an anchor tag. This is the only difference between the `Link` component and an anchor tag.

Another thing to note about our new code is that the `nav` we are rending at the top of our page is outside of our `Routes` component which means when we change pages this `nav` section will not be re-rendered as only the content in the `Routes` component will change when the URL changes

# 2. Advanced Route Definitions
This is where React Router really gets interesting. There is a lot of cool stuff you can do with routing to make more complex routes, easier to read, and overall much more functional. This can be done through five main techniques.
- Dynamic Routing
- Routing Priority
- Nested Routes
- Multiple Routes
- `useRoutes` Hook

### a. Dynamic Routing
The simplest and most common advanced feature in React Router is handling dynamic routes. In our example, let’s assume that we want to render out a component for individual books in our application. We could hardcode each of those routes, but if we have hundreds of books or the ability for users to create books then it is impossible to hardcode all these routes. Instead we need a dynamic route.

```js
  <Route path="/books" element={<BookList />} />
  <Route path="/books/:id" element={<Book />} />
```

The final route in the above example is a dynamic route that has a dynamic parameter of `:id`. Defining dynamic routes in React Router is as simple as putting a colon in front of whatever you want the dynamic part of your route to be. In our case our dynamic route will match any URL that starts with `/book` and ends with some value. For example, `/books/1`, `/books/bookName`, and `/books/literally-anything` will all match our dynamic route.

Pretty much always when you have a dynamic route like this you want to access the dynamic value in your custom component which is where the `useParams` hook comes in.

```js 
import { useParams } from "react-router-dom"

export function Book() {
  const { id } = useParams()

  return <h1>Book {id}</h1>
}
```

The `useParams` hook takes no parameters and will return an object with keys that match the dynamic parameters in your route. In our case our dynamic parameter is `:id` so the `useParams` hook will return an object that has a key of `id` and the value of that key will be the actual id in our URL. For example, if our URL was `/books/3` our page would render `Book 3`.

### b. Routing Priority
When we were just dealing with hard coded routes it was pretty easy to know which route would be rendered, but when dealing with dynamic routes it can be a bit more complicated. Take these routes for example.

```js
  <Route path="/books/:id" element={<Book />} />
  <Route path="/books/new" element={<NewBook />} />
```
If we have the URL `/books/new` which route would this match? Technically, we have two routes that match. Both `/books/:id` and `/books/new` will match since the dynamic route will just assume that `new` is the `:id` portion of the URL so React Router needs another way to determine which route to render.

In older versions of React Router whichever route was defined first would be the one that is rendered so in our case the `/books/:id` route would be rendered, which is obviously not what we want. Luckily, version 6 of React Router changed this so now React Router will use an algorithm to determine which route is most likely the one you want. In our case we obviously want to render the `/books/new` route so React Router will select that route for us. The actual way this algorithm works is very similar to CSS specificity since it will try to determine which route that matches our URL is the most specific (has the least amount of dynamic elements) and it will select that route

While we are on the topic of routing priority I also want to talk about how to create a route that matches anything.

```js
  <Route path="*" element={<NotFound />} />
```

A `*` will match anything at all which makes it perfect for things like a 404 page. A route that contains a `*` will also be less specific than anything else so you will never accidentally match a `*` route when another route would have also matched.

### c. Nested Routes
In the above example we have three routes that start with `/books` so we can nest those routes inside of each other to clean up our routes.

```js
  <Route path="/books">
    <Route index element={<BookList />} />
    <Route path=":id" element={<Book />} />
    <Route path="new" element={<NewBook />} />
  </Route>
```

All you need to do is make a parent `Route` that has the `path` prop set to the shared path for all your child `Route` components. Then inside the parent `Route` you can put all the child `Route` components. The only difference is that the `path` prop of the child `Route` components no longer includes the shared `/books` route. Also, the route for `/books` is replaced with a `Route` component that has no `path` prop, but instead has an `index` prop. All this is saying is that the path of the index `Route` is the same as the parent Route.

Now if this is all you could do with nested routes it would be only marginally useful, but the true power of nested routes comes in how it handles shared layouts


#### Shared Layouts
If you pass an `element` prop to a parent route it will render that component for every single child `Route` which means you can put a shared `nav` or other shared components on every child page with ease.

```js
// file: App.js
import {BooksLayout} from `./Booklayout`

function App(){
  // same as before
    <Route path="/books" element={<BooksLayout />}>
        <Route index element={<BookList />} />
        <Route path=":id" element={<Book />} />
        <Route path="new" element={<NewBook />} />
    </Route>
}

// file: BooksLayout.js
import {Link, Outlet} from 'react-router-dom';

export function BooksLayout(){
  return (
    <>
      <div>
        <Link to="books/1">Book 1</Link>
        <Link to="books/2">Book 2</Link>
        <Link to="books/new">New Book</Link>
      </div>
      <Outlet>
    </>
  );
}
```

The way our new code will work is whenever we match a route inside the `/book` parent `Route` it will render the `BooksLayout` component which contains our shared navigation. Then whichever child `Route` is matched will be rendered wherever the `Outlet` component is placed inside our layout component. The `Outlet` component is essentially a placeholder component that will render whatever our current page’s content is. This structure is incredibly useful and makes sharing code between routes incredibly easy.

Now the final way you can share layouts with React Router is by wrapping child `Route` components in a parent `Route` that only defines an `element` prop and no `path` prop

```js
  <Route element={<OtherLayout />}>
    <Route path="/contact" element={<Contact />} />
    <Route path="/about" element={<About />} />
  </Route>
```

This bit of code will create two routes, `/contact` and `/about`, which both are rendered inside the `OtherLayout` component. This technique of wrapping multiple `Route` components in a parent `Route` component with no `path` prop is useful if you want those routes to share a single layout even if they don’t have a similar path.

#### Outlet Context

The final important thing to know about `Outlet` components is they can take in a `context` prop which will work just like React context.

```js
// file: BookLayouts.js
<div>
  <div>
    <Link to="books/1">Book 1</Link>
    <Link to="books/2">Book 2</Link>
    <Link to="books/new">New Book</Link>
  </div>
  <Outlet context={{ hello: "world" }} />
</div>

// file: Book.js
import { useParams, useOutletContext } from "react-router-dom"

export function Book() {
  const { id } = useParams()
  const context = useOutletContext()

  return (
    <h1>
      Book {id} {context.hello}
    </h1>
  )
}
```

As you can see from this example, we are passing down a context value of `{ hello: "world" }` and then in our child component we are using the `useOutletContext` hook to access the value for our context. This is a pretty common pattern to use since often you will have shared data between all your child components which is the ideal use case for this context.

### d. Multiple `Routes`

Another incredibly powerful thing you can do with React Router is use multiple `Routes` components at the same time. This can be done as either two separate `Routes` components or as nested `Routes`.

#### Separate `Routes`

If you want to render two different sections of content that both depend on the URL of the application then you need multiple `Routes` components. This is very common if for example you have a sidebar you want to render certain content in for certain URLs and also a main page that should show specific content based on the URL.

```js
  <aside>
    <Routes>
      <Route path="/books" element={<BookSidebar />} />
    </Routes>
  </aside>

  // main Routes component
  <Routes>
    <Route path="/" element={<Home />} />
    <Route path="/books" element={<BookList />} />
  </Routes>
```

In the above example we have two `Routes`. The main `Routes` component defines all the main components for our page and then we have a secondary `Routes` inside the `aside` that will render the sidebar for our books page when we are at `/books`. This means if our URL is `/books` both of our `Routes` components will render out content since they both have a unique match for `/books` in their Routes.

Another thing that you can do with multiple `Routes` components is hardcode the `location` prop.

```js
<Routes location="/books">
  <Route path="/books" element={<BookSidebar />} />
</Routes>
```

By hardcoding a `location` prop like this we are overriding the default behavior or React Router so no matter what the URL of our page is this `Routes` component will match its `Route` as if the URL was `/books`.

#### Nested Routes
The other way to use multiple `Routes` components is to nest them inside one another. This is pretty common if you have lots of routes and want to clean up your code by moving similar routes into their own files.

```js
<Routes>
  <Route path="/" element={<Home />} />
  <Route path="/books/*" element={<BookRoutes />} />
  <Route path="*" element={<NotFound />} />
</Routes>

export function BookRoutes() {
  return (
    <Routes>
      <Route element={<BookLayout />}>
        <Route index element={<BookList />} />
        <Route path=":id" element={<Book />} />
        <Route path="new" element={<NewBook />} />
        <Route path="*" element={<NotFound />} />
      </Route>
    </Routes>
  )
}

```

Nesting `Routes` in React Router is pretty simple. All you need to do is create a new component to store your nested `Routes` this component should have a `Routes` component and inside that component should be all the `Route` components that you are matching with the parent `Route`. In our case we are moving all our `/books` routes into this `BookRoute` component. Then in the parent `Routes` component you need to define a `Route` that has a path equal to the path all your nested `Routes` share. In our case that would be `/books`. The important thing, though, is you need to end your parent `Route` path with a `*` otherwise it will not properly match the child routes.

Essentially, the code we have written says that whenever a route starts with `/book/` it should search inside the `BookRoutes` component to see if their is a `Route` that matches. This is also why we have another `*` route in `BookRoutes` so that we can ensure if our URL does not match any of the `BookRoutes` it will properly render the `NotFound` component.

### e. `useRoutes` Hook

The final thing you need to know about defining routes in React Router is that you can use a JavaScript object to define your routes instead of JSX if you prefer.

```js
    <Routes>
      <Route path="/" element={<Home />} />
      <Route path="/books">
        <Route index element={<BookList />} />
        <Route path=":id" element={<Book />} />
      </Route>
    </Routes>

    const element = useRoutes([
    {
      path: "/",
      element: <Home />,
    },
    {
      path: "/books",
      children: [
        { index: true, element: <BookList /> },
        { path: ":id", element: <Book /> },
      ],
    },
  ])

  return element
```

These two components both hae the exact same routes the only difference is how they were defined. If you do decide you want to use the `useRoutes` hook all the props that you would normally pass to your `Route` components are instead just passed as key/value pairs of an object.

## 3. Handling Navigation

Now that we know how to define our routes we need to talk about how to navigate between those routes. This section will be broken down into three sections.

- Link Navigation
- Manual Navigation
- Navigation Data

### a. Link Navigation

We have already seen the most basic form of link navigation using the `Link` component. 

```js
    <Link to="/">Home</Link>
    <Link to="/books">Books</Link>
```

These `Link` components can get a bit more complex, though. For example you can have absolute links like the above links or you can have links that are relative to the current component being rendered.

```js
    <Link to="/">Home</Link>
    <Link to="../">Back</Link>
    <Link to="edit">Edit</Link>
```

For example imagine we are in the `/books/3` route with the above links. 
- The first link will lead to the `/` route since it is an absolute route. Any route that starts with a `/` is an absolute route. 
- The second link will lead to the route `/books` since it is a relative link that goes up one level from `/books/3` to `/books`. 
- Finally, our third link will go to the `/books/3/edit` page since it will add the path in the `to` prop to the end of the current link since it is a relative link.

Besides the `to` prop, there are also 3 other props that are important to the Link component.

##### i. `replace`

The `replace` prop is a boolean that when set to `true` will cause this link to replace the current page in the browser history. Imagine you have the following browser history.

```
  /
  /books
  /books/3
```

If you click on a link that goes to the `/books/3/edit` page but it has the `replace` property set to `true` your new history will look like this.

```
  /
  /books
  /books/3/edit
```

The page your were currently on was replaced with the new page. This means that if you click the back button on the new page it will bring you back to the `/books` page instead of the `/books/3` page.

##### ii. `reloadDocument`

This prop is another boolean and if it is set to `true` your `Link` component will act like a normal anchor tag and do a full page refresh on navigation instead of just re-rendering the content inside your `Routes` component.

##### iii. `state`

This prop lets you pass data along with your `Link` that does not show up anywhere in the URL. 

#### NavLink

This component works exactly the same as the `Link` component, but it is specifically for showing active states on links, for example in nav bars. 

By default if the `to` property of a `NavLink` is the same as the URL of the current page the link will have an `active` class added to it which you can use for styling. 

If this is not enough you can instead pass a function with an `isActive` parameter to the `className`, or `style` props, or as the children of the `NavLink`. 

```js
<NavLink  to="/"
  style={({ isActive }) => ({ color: isActive ? "red" : "black" })}
>
  Home
</NavLink>
```

The `NavLink` also has one prop called `end` which is used to help with nested routing. For example, if we are on the `/books/3` page that means we are rendering the `Book` component which is nested inside our `/books` route. This means that if we have a `NavLink` with a `to` prop set to `/books` it will be considered active. 

This is because a `NavLink` is considered active if the URL matches the `to` prop of the` NavLink` or if the current `Route` being rendered is inside a parent component that has a path that matches the to prop of the NavLink. If you do not want this default behavior you can set the `end` prop to `true` which will make it so the URL of the page must exactly match the to prop of the `NavLink`.